# 常规赛：PALM病理性近视预测 8月第五名方案
## 解压数据集

In [1]:
%cd /home/aistudio/data/data93479
!unzip ./常规赛：PALM病理性近视预测.zip
!mv -f /home/aistudio/data/data93479/常规赛：PALM病理性近视预测/* /home/aistudio/data

## 清除保存的模型及结果文件
- 确认这么做的后果之前，请勿运行该代码。

In [2]:
%cd /home/aistudio/work
!rm -rf savepoint
!rm Classification_Results.csv
!mkdir -p savepoint

/home/aistudio/work
rm: cannot remove 'Classification_Results.csv': No such file or directory


## 安装PPIM
本次需要使用PPIM中的预训练模型Diet-base-224
- 持久化至external-libraries

In [3]:
#持久化ppim
!mkdir /home/aistudio/external-libraries
!pip install ppim -i https://pypi.python.org/pypi -t /home/aistudio/external-libraries

Looking in indexes: https://pypi.python.org/pypi
     |████████████████████████████████| 71kB 10.0kB/s ta 0:00:01
  Created wheel for wget: filename=wget-3.2-cp37-none-any.whl size=9675 sha256=3cb290b7c55d66311f01fc15174bd5be4efeddd6902551731eb622603e2ef4fd
  Stored in directory: /home/aistudio/.cache/pip/wheels/d1/e3/b6/e6be72d63f667cef0226c3eedff3e6658ba97d5be7d9df25dd
Successfully built wget


将安装的PPIM导入到环境

In [4]:
import sys 
sys.path.append('/home/aistudio/external-libraries')

## 准备工作
导入包，设置随机种子。

In [5]:
import os
import math
import random
import numpy as np
import pandas as pd
from PIL import Image
import paddle

def set_seed(seed):
    """sets random seed"""
    random.seed(seed)
    np.random.seed(seed)
    paddle.seed(seed)

set_seed(0)
paddle.set_device('gpu')

CUDAPlace(0)

读取训练集数据的label

In [6]:
trY=pd.read_excel('../data/Train/Classification.xlsx')
trX_list=list(trY['imgName'])
trY=np.array(trY[['Label']],dtype=np.float32)
print(len(trX_list))
print(trY.shape)

800
(800, 1)


## 生成paddle数据集类
- 对训练集图片的预处理如下：
    - resize至[224,224,3]
    - 随即旋转30度
    - 随机水平翻转
    - 随机竖直翻转
    - 改变通道位置Channel Last->Channel First
    - 输入归一化
- 对测试集图片的预处理如下：
    - resize至[224,224,3]
    - 改变通道位置Channel Last->Channel First
    - 输入归一化

In [8]:
from paddle.io import Dataset
import paddle.vision.transforms as T

class PALMTrData(Dataset):
    def __init__(self,path,pic_list,y):
        super(PALMTrData,self).__init__()
        self.path=path
        self.y=y
        self.pic_list=pic_list
        self.tf=T.Compose([
            T.Resize((224,224),interpolation='bicubic'),
            T.RandomRotation(30,interpolation='bicubic'),
            T.RandomHorizontalFlip(),
            T.RandomVerticalFlip(),
            T.ToTensor(),
            T.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
        ])
        print(len(self.pic_list))
    
    def __getitem__(self,idx):
        # X-feature
        pic_name=self.pic_list[idx]
        pic=Image.open(self.path+'/'+pic_name)
        pic_size=pic.size
        pic=self.tf(pic)
        # Y-label
        label=self.y[idx].copy()
        return pic,label

    def __len__(self):
        return len(self.pic_list)

class PALMTsData(Dataset):
    def __init__(self,path):
        super(PALMTsData,self).__init__()
        self.path=path
        self.pic_list=os.listdir(path)
        self.tf=T.Compose([
            T.Resize((224,224), interpolation='bicubic'),
            T.ToTensor(),
            T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        print(len(self.pic_list))
    
    def __getitem__(self,idx):
        pic_name=self.pic_list[idx]
        pic=Image.open(self.path+'/'+pic_name)
        pic=self.tf(pic)
        return pic,0

    def __len__(self):
        return len(self.pic_list)

tr_path='../data/Train/fundus_image'
trData=PALMTrData(path=tr_path,pic_list=trX_list,y=trY)
ts_path='../data/PALM-Testing400-Images'
tsData=PALMTsData(path=ts_path)

800
400


## 构建模型
- 模型使用预训练模型resnet50 + Diet_base_224为主体，二者直接并联，输出为[1000+1000]
- 下游任务分类器结构为2层全连接网络(2000->100->2)，以relu激活，并辅以Dropout=0.2防止过拟合
- 下游任务被设定为二分类问题

In [9]:
from paddle import nn
import paddle.nn.functional as F
from ppim import deit_b_distilled
from paddle.vision.models import resnet50

class PALM(nn.Layer):
    def __init__(self):
        super(PALM,self).__init__()
        self.deit=deit_b_distilled(pretrained=True)
        self.resnet=resnet50(pretrained=True)
        self.do1=nn.Dropout(p=0.2)
        self.lr1=nn.Linear(in_features=2000, out_features=100)
        self.do2=nn.Dropout(p=0.2)
        self.lr2=nn.Linear(in_features=100, out_features=1)

    def forward(self,x):
        x1=self.deit(x)
        x2=self.resnet(x)
        x=paddle.concat([x1,x2],axis=-1)
        x=F.relu(x)
        x=self.do1(x)
        x=self.lr1(x)
        x=F.relu(x)
        x=self.do2(x)
        x=self.lr2(x)
        x=F.sigmoid(x)
        return x

palm=PALM()

100%|██████████| 151272/151272 [00:02<00:00, 63011.04it/s]


## 模型训练参数
- 优化器：adam
- Loss：二分类使用BCELoss
- 学习率：固定5e-5

In [10]:
from paddle.optimizer import Adam
from paddle.nn import BCELoss
from paddle.metric import Accuracy

inputs=paddle.static.InputSpec([-1, 3, 224, 224], dtype='float32', name='input')
label=paddle.static.InputSpec([-1, 1], dtype='float32', name='label')
model=paddle.Model(palm, inputs, label)
model.summary()
model.prepare(optimizer=Adam(learning_rate=5e-5,parameters=model.parameters()),loss=BCELoss(),metrics=Accuracy())

----------------------------------------------------------------------------------------
        Layer (type)             Input Shape          Output Shape         Param #    
          Conv2D-5            [[1, 3, 224, 224]]    [1, 768, 14, 14]       590,592    
        PatchEmbed-1          [[1, 3, 224, 224]]     [1, 196, 768]            0       
         Dropout-1             [[1, 198, 768]]       [1, 198, 768]            0       
        LayerNorm-1            [[1, 198, 768]]       [1, 198, 768]          1,536     
          Linear-1             [[1, 198, 768]]       [1, 198, 2304]       1,771,776   
         Dropout-2           [[1, 12, 198, 198]]   [1, 12, 198, 198]          0       
          Linear-2             [[1, 198, 768]]       [1, 198, 768]         590,592    
         Dropout-3             [[1, 198, 768]]       [1, 198, 768]            0       
        Attention-1            [[1, 198, 768]]       [1, 198, 768]            0       
         Identity-1            [[1, 198, 

## 训练模型
- 训练、预测批大小：16
- 训练回合：50
- 数据异步读取：2个进程
- 训练集：全部训练数据
- 验证集：无

In [11]:
model.fit(train_data=trData,batch_size=16,epochs=50,verbose=1,num_workers=2)

The loss value printed in the log is the current step, and the metric is the average value of previous steps.
Epoch 1/50


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/tensor/creation.py:125: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if data.dtype == np.object:
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/tensor/creation.py:125: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if data.dtype == np.object:
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/utils.py:77: DeprecationWarning: Using or impor

step 50/50 [==============================] - loss: 0.1334 - acc: 0.4700 - 956ms/step        
Epoch 2/50


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/tensor/creation.py:125: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if data.dtype == np.object:
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/tensor/creation.py:125: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if data.dtype == np.object:


step 50/50 [==============================] - loss: 0.0500 - acc: 0.4700 - 927ms/step        
Epoch 3/50


...

step 50/50 [==============================] - loss: 0.0593 - acc: 0.4700 - 966ms/step            
Epoch 49/50


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/tensor/creation.py:125: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if data.dtype == np.object:
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/tensor/creation.py:125: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if data.dtype == np.object:


step 50/50 [==============================] - loss: 0.0360 - acc: 0.4700 - 923ms/step            
Epoch 50/50


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/tensor/creation.py:125: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if data.dtype == np.object:
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/tensor/creation.py:125: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if data.dtype == np.object:


step 50/50 [==============================] - loss: 5.6150e-05 - acc: 0.4700 - 1s/step          


## 模型预测
- 对测试集数据预测
- 预测批大小：16
- 预测数据异步读取：2个进程

In [12]:
preds=model.predict(tsData,batch_size=16,num_workers=2)

Predict begin...


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/tensor/creation.py:125: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if data.dtype == np.object:
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/tensor/creation.py:125: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if data.dtype == np.object:


step 25/25 [==============================] - 859ms/step         
Predict samples: 400


## 保存预测结果
- 记得按文件名重新排序

In [13]:
pred=np.squeeze(np.array(preds)).reshape((400,))
pred_list=os.listdir(ts_path)
print(pred.shape)
ans=pd.DataFrame({'FileName':pred_list,'PM Risk':pred})
ans=ans.sort_values(by='FileName',ignore_index=True)
ans.to_csv('./Classification_Results.csv',index=0)

(400,)


## 保存模型

In [14]:
model.save(path='./savepoint/final',training=False)

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/math_op_patch.py:322: UserWarning: /tmp/tmp1rnw_m37.py:8
The behavior of expression A + B has been unified with elementwise_add(X, Y, axis=-1) from Paddle 2.0. If your code works well in the older versions but crashes in this version, try to use elementwise_add(X, Y, axis=0) instead of A + B. This transitional warning will be dropped in the future.
  op_type, op_type, EXPRESSION_MAP[method_name]))
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/math_op_patch.py:322: UserWarning: /tmp/tmp1rnw_m37.py:11
The behavior of expression A + B has been unified with elementwise_add(X, Y, axis=-1) from Paddle 2.0. If your code works well in the older versions but crashes in this version, try to use elementwise_add(X, Y, axis=0) instead of A + B. This transitional warning will be dropped in the future.
  op_type, op_type, EXPRESSION_MAP[method_name]))
/opt/conda/envs/python3

## 注意事项与特殊说明
由于GitHub对大文件上传限制，模型训练参数checkpoint无法上传。
- 完整repo详见AI studio项目：https://aistudio.baidu.com/aistudio/projectdetail/2318275